In [ ]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT
!pip install tensorflow==2.0.0-beta0
!pip install pythainlp

import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 87.9MB 49kB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 3.1MB 54.0MB/s 
     |████████████████████████████████| 501kB 50.7MB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
ERROR: Operation cancelled by user
     |████████████████████████████████| 8.9MB 4.1MB/s 
     |████████████████████████████████| 747kB 56.6MB/s 
2.3.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [ ]:
!wget --no-check-certificate \
    https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip \
    -O /tmp/review_dataset.zip

%cd /tmp
!unzip /tmp/review_dataset.zip
!ls


In [ ]:
!head w_review_train.csv

In [ ]:
import csv
import numpy as np
from pythainlp.corpus.common import thai_stopwords
from pythainlp import word_tokenize

stopwords = thai_stopwords();
print(stopwords)

sentences = []
labels = []

count = 0;

with open("/tmp/w_review_train.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        count+=1
        labels.append(float(row[1])-1)
        sentence = ' '.join(word_tokenize(row[0]))
        for word in stopwords:
            token = " " + word + " "
            sentence = sentence.replace(token, " ")
            sentence = sentence.replace("  ", " ")
        sentences.append(sentence)
        if (count % 100 == 0):
          print(count)
        if (count > 5000):
          break;

print(len(sentences))

frozenset({'ครั้งหลัง', 'ยังคง', 'มันๆ', 'ทําให้', 'จน', 'แค่เพียง', 'หากแม้นว่า', 'อย่างเดียว', 'มักจะ', 'แต่ทว่า', 'ด้วยเหตุนี้', 'คราวหลัง', 'ค่อยๆ', 'ก็', 'คล้ายกันกับ', 'เป็นอันๆ', 'ก่อน', 'ถึงจะ', 'ทุก', 'ยืนยาว', 'ทุกทาง', 'ฝ่ายใด', 'นับ', 'ยิ่งใหญ่', 'กลับ', 'แต่ละ', 'นาง', 'ตลอดทั่วทั้ง', 'รับรอง', 'ฉัน', 'นิดๆ', 'เพียง', 'ใดๆ', 'ตรง', 'คราวหน้า', 'เสียแล้ว', 'ค่อนมาทาง', 'เมื่อครั้งก่อน', 'ก็ได้', 'ย่อย', 'ดั่งเคย', 'นู่น', 'มี', 'ตลอดทั่วถึง', 'แม้แต่', 'พบ', 'กลุ่มๆ', 'ในระหว่าง', 'เปลี่ยนแปลง', 'ว่า', 'ประการใด', 'หารือ', 'ตลอดทั้ง', 'ได้มา', 'เผื่อ', 'ไม่', 'เสียจน', 'เสร็จกัน', 'อาจจะ', 'เชื่อ', 'ในช่วง', 'ให้แก่', 'กระนั้น', 'พอดี', '\ufeffๆ', 'จับ', 'ทุกที', 'ยิ่งจะ', 'สมัยนี้', 'พอๆ', 'มุ่ง', 'ฉะนั้น', 'เรียบ', 'แต่นั้น', 'จวน', 'สูงๆ', 'อีก', 'แก้ไข', 'ตลอดทั่ว', 'ครั้งที่', 'นัก', 'เลย', 'จัดงาน', 'เห็นควร', 'ยิ่งขึ้นไป', 'นี้แหล่', 'ที่ซึ่ง', 'นอกจากที่', 'เมื่อก่อน', 'เช่นเดียวกับ', 'คราวที่', 'ล้วนแต่', 'นํา', 'ที่ได้', 'หาความ', 'เสมือนกับ', 'ต่าง', 'ตลอดระยะเวล

In [ ]:
sentences = np.array(sentences);       
labels = np.array(labels);       

training_sentences = sentences[:4000]
training_labels = labels[:4000]

testing_sentences = sentences[4000:]
testing_labels = labels[4000:]

In [ ]:
training_labels

array([2., 3., 2., ..., 4., 2., 3.])

In [ ]:
vocab_size = 3000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [ ]:
word_index

{'<OOV>': 1,
 'ร้าน': 2,
 'อร่อย': 3,
 'ทาน': 4,
 'ดี': 5,
 'รสชาติ': 6,
 'กิน': 7,
 'อาหาร': 8,
 'เมนู': 9,
 'สั่ง': 10,
 'ราคา': 11,
 'คน': 12,
 'บาท': 13,
 'ชอบ': 14,
 'เนื้อ': 15,
 'ผม': 16,
 'ลอง': 17,
 'หมู': 18,
 'จาน': 19,
 'น้ำ': 20,
 'หวาน': 21,
 'ปลา': 22,
 'ดู': 23,
 'นั่ง': 24,
 'รส': 25,
 'เค้า': 26,
 'บรรยากาศ': 27,
 'ไม่ได้': 28,
 'เลือก': 29,
 'นะคะ': 30,
 '2': 31,
 'ทอด': 32,
 'บริการ': 33,
 'นุ่ม': 34,
 'กุ้ง': 35,
 'สำหรับ': 36,
 'เหมือน': 37,
 'แนะนำ': 38,
 'ที่นี่': 39,
 'ใส่': 40,
 'หอม': 41,
 'สาขา': 42,
 'ขาย': 43,
 'โต๊ะ': 44,
 'ผัด': 45,
 '3': 46,
 'ข้าว': 47,
 'กรอบ': 48,
 'แพง': 49,
 'ตัว': 50,
 'ชิ้น': 51,
 'สด': 52,
 'ไก่': 53,
 'ซุป': 54,
 'ทำ': 55,
 '1': 56,
 'เค้ก': 57,
 'หน้า': 58,
 '5': 59,
 'กก': 60,
 'พนักงาน': 61,
 'ไม่มี': 62,
 'ย่าง': 63,
 'ร้านอาหาร': 64,
 'ใช้ได้': 65,
 'ก๋วยเตี๋ยว': 66,
 'รอ': 67,
 'ชิม': 68,
 'ชา': 69,
 'เส้น': 70,
 'ร้อน': 71,
 'น้ำจิ้ม': 72,
 '4': 73,
 'ซอย': 74,
 'เดิน': 75,
 '10': 76,
 'ซอส': 77,
 'เผ็ด': 78,
 'เพื่อน': 

In [ ]:
len(word_index)

17138

In [ ]:
padded

array([[   0,    0,    0, ..., 2696, 1079,   49],
       [   0,    0,    0, ...,    3,   14,  298],
       [   0,    0,    0, ...,    1,    1,  163],
       ...,
       [   0,    0,    0, ...,    7,    3,    1],
       [   0,    0,    0, ...,  163,  539,    1],
       [   0,    0,    0, ...,    1,    1,    1]], dtype=int32)

In [ ]:
padded.shape

(4000, 120)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(padded[2])
print(decode_review(padded[2]))
print(training_sentences[2])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0 1255   52   94
 1535    2   43 1255   52   97   43 1834    3    2  743  779   11    1
  184   13  103    7  242    6    5  307  656 1490   40 1716   86   72
   82  524  492  171    1  694  482   41  150  612  171   44   24    7
    2  603   73  163 1781   36    8   97    1 1781 1836  587   55 1255
   40    1 1618    1   16    1    1  163]
? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ปอเปี๊ยะ สด รู้สึก หากิน ร้าน ขาย ปอเปี๊ยะ สด เจอ ขาย ภัตตาคาร อร่อย ร้าน สะพาน เหลือง ราคา <OOV> 30 บาท ซื้อ กิน ที่มา รสชาติ ดี โดยเฉพาะ เต้าหู้ กุนเชียง ใส่ จุใจ ราด น้ำจิ้ม โอเค ถึงกับ สุดยอด เสิร์ฟ <OOV> เครื่องเคียง ต้น หอม พริก เขียว เสิร์ฟ โต๊ะ นั่ง กิน ร้าน อารมณ์ 4 ดาว คุณป

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           48000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                19210     
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 55        
Total params: 67,265
Trainable params: 67,265
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 10
model.fit(padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))

Train on 4000 samples, validate on 1001 samples
Epoch 1/10
4000/4000 [==============================] - 0s 105us/sample - loss: 7.5778e-04 - accuracy: 1.0000 - val_loss: 2.9395 - val_accuracy: 0.4156
Epoch 2/10
4000/4000 [==============================] - 0s 97us/sample - loss: 6.8137e-04 - accuracy: 1.0000 - val_loss: 2.9743 - val_accuracy: 0.4166
Epoch 3/10
4000/4000 [==============================] - 0s 96us/sample - loss: 6.1354e-04 - accuracy: 1.0000 - val_loss: 2.9901 - val_accuracy: 0.4146
Epoch 4/10
4000/4000 [==============================] - 0s 102us/sample - loss: 5.5384e-04 - accuracy: 1.0000 - val_loss: 3.0047 - val_accuracy: 0.4106
Epoch 5/10
4000/4000 [==============================] - 0s 98us/sample - loss: 5.0126e-04 - accuracy: 1.0000 - val_loss: 3.0505 - val_accuracy: 0.4156
Epoch 6/10
4000/4000 [==============================] - 0s 96us/sample - loss: 4.5484e-04 - accuracy: 1.0000 - val_loss: 3.0887 - val_accuracy: 0.4166
Epoch 7/10
4000/4000 [======================

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(3000, 16)


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  if (word and len(word)>1):
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')